In [1]:
# -*- coding: utf-8 -*-
"""keras_bert_classification_tpu.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1kzGClSAy-SPo_dvgKPIgP0oJCFdZ_8Pa
"""

# @title Preparation
!pip install -q keras-bert
!wget -q https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip -o uncased_L-12_H-768_A-12.zip

# @title Constants

SEQ_LEN = 128
BATCH_SIZE = 32 # 64 seems to be the maximum possible on Kaggle
EPOCHS = 5
LR = 1e-4

# @title Environment
import os

pretrained_path = 'uncased_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

# TF_KERAS must be added to environment variables in order to use TPU
os.environ['TF_KERAS'] = '1'

# @title Load Basic Model
import codecs
from keras_bert import load_trained_model_from_checkpoint

token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)

model = load_trained_model_from_checkpoint(
    config_path,
    checkpoint_path,
    training=True,
    trainable=True,
    seq_len=SEQ_LEN,
)

# @title Download IMDB Data
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

84131840/84125825 [==============================] - 1s 0us/step


In [2]:
# @title Convert Data to Array
import os
import numpy as np
from tqdm import tqdm
from keras_bert import Tokenizer

tokenizer = Tokenizer(token_dict)


def load_data(path):
    global tokenizer
    indices, sentiments = [], []
    for folder, sentiment in (('neg', 0), ('pos', 1)):
        folder = os.path.join(path, folder)
        for name in tqdm(os.listdir(folder)):
            with open(os.path.join(folder, name), 'r') as reader:
                  text = reader.read()
            ids, segments = tokenizer.encode(text, max_len=SEQ_LEN)
            indices.append(ids)
            sentiments.append(sentiment)
    items = list(zip(indices, sentiments))
    np.random.shuffle(items)
    indices, sentiments = zip(*items)
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)], np.array(sentiments)
  
  
  
train_path = os.path.join(os.path.dirname(dataset), 'aclImdb', 'train')
test_path = os.path.join(os.path.dirname(dataset), 'aclImdb', 'test')


train_x, train_y = load_data(train_path)
test_x, test_y = load_data(test_path)



100%|██████████| 12500/12500 [00:38<00:00, 322.08it/s]


In [3]:
print("train_x list details:")
print(len(train_x))
print(train_x[:5])
print(train_y[:5])
print(train_y.shape)

# @title Build Custom Model
from tensorflow.python import keras
from keras_bert import AdamWarmup, calc_train_steps

inputs = model.inputs[:2]
dense = model.get_layer('NSP-Dense').output
outputs = keras.layers.Dense(units=2, activation='softmax')(dense)

decay_steps, warmup_steps = calc_train_steps(
    train_y.shape[0],
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
)

model = keras.models.Model(inputs, outputs)

#freeze some layers
for layer in model.layers:
    layer.trainable = False

model.layers[-1].trainable = True
model.layers[-2].trainable = True
model.layers[-3].trainable = True

model.compile(
    AdamWarmup(decay_steps=decay_steps, warmup_steps=warmup_steps, lr=LR),
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'],
)

# @title Initialize Variables
import tensorflow as tf
import tensorflow.keras.backend as K

sess = K.get_session()
uninitialized_variables = set([i.decode('ascii') for i in sess.run(tf.report_uninitialized_variables())])
init_op = tf.variables_initializer(
    [v for v in tf.global_variables() if v.name.split(':')[0] in uninitialized_variables]
)
sess.run(init_op)

train_x list details:
2
[array([[  101,  1000, 23564, ...,  2006,  2019,   102],
       [  101,  2045,  2024, ...,  2008,  2009,   102],
       [  101,  2019,  2256, ...,  5006,  7664,   102],
       ...,
       [  101,  2023,  2003, ...,     0,     0,     0],
       [  101,  3475,  1005, ...,  2370,  1999,   102],
       [  101,  2023,  2143, ...,  2612,  1010,   102]]), array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])]
[1 0 1 1 1]
(25000,)


In [4]:
from keras_bert import get_custom_objects
# @title Fit

with tf.keras.utils.custom_object_scope(get_custom_objects()):
    model.fit(
        train_x,
        train_y,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
    )

# @title Predict

with tf.keras.utils.custom_object_scope(get_custom_objects()):
    predicts = model.predict(test_x, verbose=True).argmax(axis=-1)

# @title Accuracy

print(np.sum(test_y == predicts) / test_y.shape[0])

Epoch 1/5
25000/25000 [==============================] - 120s 5ms/sample - loss: 0.5332 - sparse_categorical_accuracy: 0.7324
Epoch 2/5
25000/25000 [==============================] - 118s 5ms/sample - loss: 0.4597 - sparse_categorical_accuracy: 0.7839
Epoch 3/5
25000/25000 [==============================] - 118s 5ms/sample - loss: 0.4495 - sparse_categorical_accuracy: 0.7923
Epoch 4/5
25000/25000 [==============================] - 118s 5ms/sample - loss: 0.4433 - sparse_categorical_accuracy: 0.7946
Epoch 5/5
25000/25000 [==============================] - 113s 5ms/sample
0.80892


In [5]:
!ls

__notebook_source__.ipynb  uncased_L-12_H-768_A-12  uncased_L-12_H-768_A-12.zip
